In [10]:
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split


#data_root="classes_dataset"
images = "original_images"
masks = "label_images_semantic"


In [11]:
!unzip label_images_semantic.zip


Archive:  label_images_semantic.zip
   creating: label_images_semantic/
  inflating: label_images_semantic/000.png  
  inflating: label_images_semantic/001.png  
  inflating: label_images_semantic/002.png  
  inflating: label_images_semantic/003.png  
  inflating: label_images_semantic/004.png  
  inflating: label_images_semantic/005.png  
  inflating: label_images_semantic/006.png  
  inflating: label_images_semantic/008.png  
  inflating: label_images_semantic/011.png  
  inflating: label_images_semantic/013.png  
  inflating: label_images_semantic/014.png  
  inflating: label_images_semantic/015.png  
  inflating: label_images_semantic/016.png  
  inflating: label_images_semantic/018.png  
  inflating: label_images_semantic/019.png  
  inflating: label_images_semantic/021.png  
  inflating: label_images_semantic/022.png  
  inflating: label_images_semantic/023.png  
  inflating: label_images_semantic/026.png  
  inflating: label_images_semantic/028.png  
  inflating: label_images_se

In [12]:
!unzip original_images.zip

Archive:  original_images.zip
replace original_images/000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace original_images/000.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: original_images/000.png  
  inflating: original_images/001.png  
  inflating: original_images/002.png  
  inflating: original_images/003.png  
  inflating: original_images/004.png  
  inflating: original_images/005.png  
  inflating: original_images/006.png  
  inflating: original_images/008.png  
  inflating: original_images/011.png  
  inflating: original_images/013.png  
  inflating: original_images/014.png  
  inflating: original_images/015.png  
  inflating: original_images/016.png  
  inflating: original_images/018.png  
  inflating: original_images/019.png  
  inflating: original_images/021.png  
  inflating: original_images/022.png  
  inflating: original_images/023.png  
  inflating: original_images/026.png  
  inflating: original_images/028.png  
  inflating: or

In [13]:
class SegmentationDataset(Dataset):
    def __init__(self, images, masks, img_size=(572, 572)):
        self.images = images
        self.masks = masks
        self.img_size = img_size
        self.rgb_to_class = {
            (155, 38, 182): 0,
            (14, 135, 204): 1,
            (124, 252, 0): 2,
            (255, 20, 147): 3,
            (169, 169, 169): 4
        }

    def __len__(self):
        return len(self.images)

    def rgb_to_mask(self, rgb):
        h, w, _ = rgb.shape
        mask = np.zeros((h, w), dtype=np.int64)
        for rgb_val, class_idx in self.rgb_to_class.items():
            color = np.array(rgb_val, dtype=np.uint8)
            class_mask = cv2.inRange(rgb, color, color)
            mask[class_mask > 0] = class_idx
        return mask

    def __getitem__(self, idx):
        img = cv2.imread(self.images[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, self.img_size)
        img = torch.from_numpy(img / 255.0).permute(2, 0, 1).float()

        msk = cv2.imread(self.masks[idx])
        msk = cv2.cvtColor(msk, cv2.COLOR_BGR2RGB)
        msk = cv2.resize(msk, (388, 388), interpolation=cv2.INTER_NEAREST)
        msk = torch.from_numpy(self.rgb_to_mask(msk)).long()

        return img, msk


In [14]:
def get_data_loaders(img_dir, mask_dir, batch_size=2):
    imgs = sorted([os.path.join(img_dir, f) for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg'))])
    masks = sorted([os.path.join(mask_dir, f) for f in os.listdir(mask_dir) if f.endswith(('.png', '.jpg'))])

    train_imgs, val_imgs, train_masks, val_masks = train_test_split(imgs, masks, test_size=0.2, random_state=42)

    train_set = SegmentationDataset(train_imgs, train_masks)
    val_set = SegmentationDataset(val_imgs, val_masks)

    train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader


In [15]:
def conv_block(in_c, out_c):
    block = nn.Sequential(
        nn.Conv2d(in_c,out_c,kernel_size=3),
        nn.ReLU(),
        nn.Conv2d(out_c,out_c,kernel_size=3),
        nn.ReLU()
    )
    return block

def crop_img(tensor,target_tensor):
    target_size=target_tensor.size()[2]
    tensor_size=tensor.size()[2]
    delta=tensor_size-target_size
    delta=delta//2
    return tensor[:,:, delta:tensor_size-delta,delta:tensor_size-delta]


In [16]:
class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        self.Max_Pool=nn.MaxPool2d(kernel_size=2,stride=2)
        self.down_conv1 = conv_block(in_channels, 64)
        self.down_conv2 = conv_block(64, 128)
        self.down_conv3 = conv_block(128, 256)
        self.down_conv4 = conv_block(256, 512)
        self.down_conv5 = conv_block(512, 1024)
        self.Trans_conv1=nn.ConvTranspose2d(1024,512,kernel_size=2,stride=2)
        self.up_conv1 = conv_block(1024, 512)
        self.Trans_conv2=nn.ConvTranspose2d(512,256,kernel_size=2,stride=2)
        self.up_conv2 = conv_block(512, 256)
        self.Trans_conv3=nn.ConvTranspose2d(256,128,kernel_size=2,stride=2)
        self.up_conv3 = conv_block(256, 128)
        self.Trans_conv4=nn.ConvTranspose2d(128,64,kernel_size=2,stride=2)
        self.up_conv4 = conv_block(128, 64)
        self.out = nn.Conv2d(64,out_channels,kernel_size=1)
    def forward(self,x):
       #encoder
       x1=self.down_conv1(x)
       x2=self.Max_Pool(x1)
       x3=self.down_conv2(x2)
       x4=self.Max_Pool(x3)
       x5=self.down_conv3(x4)
       x6=self.Max_Pool(x5)
       x7=self.down_conv4(x6)
       x8=self.Max_Pool(x7)
       x9=self.down_conv5(x8)
       print(x9.size())

       #decoder
       x=self.Trans_conv1(x9)
       y=crop_img(x7,x)
       x=self.up_conv1(torch.cat((x,y),1))

       x=self.Trans_conv2(x)
       y=crop_img(x5,x)
       x=self.up_conv2(torch.cat((x,y),1))

       x=self.Trans_conv3(x)
       y=crop_img(x3,x)
       x=self.up_conv3(torch.cat((x,y),1))

       x=self.Trans_conv4(x)
       y=crop_img(x1,x)
       x=self.up_conv4(torch.cat((x,y),1))

       x=self.out(x)
       print(x.size())
       return x






In [17]:
import torch.optim as optim
from tqdm import tqdm

class Trainer:
    def __init__(self, model, device, lr=1e-4):
        self.model = model.to(device)
        self.device = device
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(model.parameters(), lr=lr)
        self.best_loss = float('inf')

    def train_epoch(self, loader):
        self.model.train()
        epoch_loss = 0
        for images, masks in tqdm(loader, desc="Train"):
            images, masks = images.to(self.device), masks.to(self.device)
            self.optimizer.zero_grad()
            outputs = self.model(images)
            loss = self.criterion(outputs, masks)
            loss.backward()
            self.optimizer.step()
            epoch_loss += loss.item()
        return epoch_loss / len(loader)

    def validate(self, loader):
        self.model.eval()
        val_loss = 0
        with torch.no_grad():
            for images, masks in tqdm(loader, desc="Val"):
                images, masks = images.to(self.device), masks.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, masks)
                val_loss += loss.item()
        return val_loss / len(loader)

    def train(self, train_loader, val_loader, epochs=20):
        for epoch in range(epochs):
            print(f"\nEpoch {epoch+1}/{epochs}")
            train_loss = self.train_epoch(train_loader)
            val_loss = self.validate(val_loader)
            print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            if val_loss < self.best_loss:
                self.best_loss = val_loss
                torch.save(self.model.state_dict(), "best_unet_model.pth")
                print(" Saved best model.")


In [18]:
if __name__ == "__main__":
    image_dir= images
    mask_dir= masks

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = UNet(in_channels=3, out_channels=5)
    trainer = Trainer(model, device)

    train_loader, val_loader = get_data_loaders(image_dir, mask_dir, batch_size=2)
    trainer.train(train_loader, val_loader, epochs=50)



Epoch 1/50


Train:   0%|          | 0/160 [00:00<?, ?it/s]

torch.Size([2, 1024, 28, 28])
torch.Size([2, 5, 388, 388])


Train:   1%|          | 1/160 [00:01<03:54,  1.47s/it]

torch.Size([2, 1024, 28, 28])
torch.Size([2, 5, 388, 388])


...

Val:  98%|█████████▊| 39/40 [00:09<00:00,  4.26it/s]

torch.Size([2, 1024, 28, 28])
torch.Size([2, 5, 388, 388])


Val: 100%|██████████| 40/40 [00:09<00:00,  4.22it/s]

torch.Size([2, 1024, 28, 28])
torch.Size([2, 5, 388, 388])
Train Loss: 0.6362 | Val Loss: 0.7156


 Saved best model.


In [24]:
import shutil

# Change this to your actual username if needed
desktop_path = r"C:\Users\saart\OneDrive\Desktop\best_unet_model.pth"

# Move file to Desktop
shutil.move(r"C:\Users\saart\Desktop\best_unet_model.pth", desktop_path)

print(f"✅ Model successfully moved to: {desktop_path}")

✅ Model successfully moved to: C:\Users\saart\OneDrive\Desktop\best_unet_model.pth
